In [ ]:
%config InlineBackend.figure_format='retina'

In [ ]:
#import time
#import os
#
#while True:
#    
#    if os.path.exists(
#        '/GPUData_xingjie/SCMG/contrastive_embedding_training/edges/inter_dataset/Tabula_Sapiens_HS_2022_all_AND_Yu_MultiTissue_HS_2021_all.parquet'
#    ):
#        break
#
#    time.sleep(10)

In [ ]:
import os
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import anndata
import scanpy as sc
import datasets

In [ ]:
output_path = '/GPUData_xingjie/SCMG/contrastive_embedding_training/training_dataset/'

In [ ]:
exp_dataset = datasets.load_from_disk(os.path.join(output_path, 'combined_exp'))
exp_dataset

In [ ]:
cell_id_df = pd.DataFrame({
    'cell_id' : exp_dataset['cell_id'],
    'dataset_id' : exp_dataset['dataset_id']
})
cell_id_df.to_parquet(os.path.join(output_path, 'cell_id.parquet'))

cell_id_to_idx_map = {cell_id: idx for idx, cell_id in enumerate(exp_dataset['cell_id'])}

cell_id_df

In [ ]:
edge_file_paths = [
    '/GPUData_xingjie/SCMG/contrastive_embedding_training/edges/inter_dataset',
    '/GPUData_xingjie/SCMG/contrastive_embedding_training/edges/intra_dataset_core',
    '/GPUData_xingjie/SCMG/contrastive_embedding_training/edges/intra_integration'
]

edge_df_list = []

for edge_file_path in edge_file_paths:
    for f in os.listdir(edge_file_path):
        if f.endswith('.parquet'):
            edge_df = pd.read_parquet(os.path.join(edge_file_path, f))
            edge_df_list.append(edge_df)

edge_df = pd.concat(edge_df_list, axis=0)
edge_df

In [ ]:
all_datasets = np.unique(list(edge_df['dataset_ref']) + list(edge_df['dataset_query']))
dataset_id_map = {dataset_id: idx for idx, dataset_id in enumerate(all_datasets)}
with open(os.path.join(output_path, 'dataset_id_map.json'), 'w') as f:
    json.dump(dataset_id_map, f)

In [ ]:
#edge_files = [
#    '/GPUData_xingjie/SCMG/contrastive_embedding_training/edges/intra_dataset/Suo_ImmuneDev_HS_2022_all.parquet',
#    '/GPUData_xingjie/SCMG/contrastive_embedding_training/edges/intra_integration/Suo_ImmuneDev_HS_2022_all_intra_0.parquet',
#    '/GPUData_xingjie/SCMG/contrastive_embedding_training/edges/intra_integration/Suo_ImmuneDev_HS_2022_all_intra_1.parquet',
#    '/GPUData_xingjie/SCMG/contrastive_embedding_training/edges/intra_integration/Suo_ImmuneDev_HS_2022_all_intra_2.parquet',
#]
#
#edge_df_list = []
#
#for f in edge_files:
#    if f.endswith('.parquet'):
#        edge_df = pd.read_parquet(f)
#        edge_df_list.append(edge_df)
#
#edge_df = pd.concat(edge_df_list, axis=0)
#edge_df

In [ ]:
edge_df['cell_ref_idx'] = edge_df['cell_ref'].map(cell_id_to_idx_map)
edge_df['cell_query_idx'] = edge_df['cell_query'].map(cell_id_to_idx_map)

from datasets import Dataset

edge_dataset = Dataset.from_dict({
            'cell_ref_idx': edge_df['cell_ref_idx'].values,
            'cell_query_idx': edge_df['cell_query_idx'].values,
            'dataset_ref': edge_df['dataset_ref'].values,
            'dataset_query': edge_df['dataset_query'].values
                                     })

edge_dataset.save_to_disk(os.path.join(output_path, 'edge_dataset'))

In [ ]:
#edge_df.iloc[:10].to_csv(os.path.join(output_path, 'done.csv'), index=False)